### <center>  Задание 9.5

In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx', engine='openpyxl') 

In [3]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [4]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [5]:
y_predict_train = selector.predict(X_train) # Делаем предсказание на тренировочной выборке
y_predict_test = selector.predict(X_test) # Делаем предсказание на тестовой выборке
print('MAE on train: {:.3f}'.format(mean_absolute_error(y_train, y_predict_train)))
print('MAE on test: {:.3f}'.format(mean_absolute_error(y_test, y_predict_test)))


MAE on train: 5283.260
MAE on test: 5096.570


In [6]:
y_train_predict_r2 = selector.predict(X_train) # Делаем предсказание на тренировочной выборке
y_test_predict_r2 = selector.predict(X_test) # Делаем предсказание на тестовой выборке
print('Train R^2: {:.3f}'.format(r2_score(y_train, y_train_predict_r2)))
print('Test R^2: {:.3f}'.format(r2_score(y_test, y_test_predict_r2)))

Train R^2: 0.480
Test R^2: 0.576


Качество модели на тесте МАЕ показало не очень хороший результат, так как погрешность в 5096 долларов я считаю много.

Качество модели на тесте R^2, показало 0.576, это минимум который допустим, но так же очень маленький показатель.

SelectKBest

In [7]:
# Обьявляем метод К лучьших переменных
selectorKB = SelectKBest(f_regression, k=3)

In [8]:
# Отбираем 3 признака
selectorKB.fit(X_train, y_train)
selectorKB.get_feature_names_out()

SelectKBest(k=3, score_func=<function f_regression at 0x000002E0F2A6E5E0>)

array(['year', 'cylinders', 'odometer'], dtype=object)

In [9]:
# Преобразуем признаки
X_train_KB= selectorKB.fit_transform(X_train, y_train)
X_test_KB = selectorKB.fit_transform(X_test, y_test)

In [10]:
# Обучаем модель
Lreg = LinearRegression()
Lreg.fit(X_train_KB, y_train)


LinearRegression()

In [11]:
y_train_predict = Lreg.predict(X_train_KB) # Делаем предсказание на тренировочной выборке
y_test_predict = Lreg.predict(X_test_KB) # Делаем предсказание на тестовой выборке

In [12]:
# Считаем метрики
print('Train R^2: {:.3f}'.format((r2_score(y_train, y_train_predict))))
print("Test R^2: {:.3f}".format((r2_score(y_test, y_test_predict)))+('\n'))
print('Train MAE: {:.3f}'.format((mean_absolute_error(y_train, y_train_predict))))
print('Test MAE: {:.3f}'.format((mean_absolute_error(y_test, y_test_predict))))



Train R^2: 0.511
Test R^2: 0.602

Train MAE: 4896.205
Test MAE: 4708.946


In [20]:
# Для сравнения на одной строке метрики с разными фильтрами 

    
print('MAE on train with RFE: {:.3f}'.format(mean_absolute_error(y_train, y_predict_train)), 'and', round(mean_absolute_error(y_train, y_train_predict), 3) , 'MAE on train with SelectKBest')
print('MAE on test with RFE: {:.3f}'.format(mean_absolute_error(y_test, y_predict_test)), 'and', round(mean_absolute_error(y_test, y_test_predict), 3) , 'MAE on test with SelectKBest')
print('Train R^2 with RFE: {:.3f}'.format(r2_score(y_train, y_train_predict_r2)), 'and', round(r2_score(y_train, y_train_predict), 3) , 'R^2 on train with SelectKBest')
print('Test R^2 with RFE: {:.3f}'.format(r2_score(y_test, y_test_predict_r2)), 'and', round(r2_score(y_test, y_test_predict), 3) , 'R^2 on test with SelectKBest')

MAE on train with RFE: 5283.260 and 4896.205 MAE on train with SelectKBest
MAE on test with RFE: 5096.570 and 4708.946 MAE on test with SelectKBest
Train R^2 with RFE: 0.480 and 0.511 R^2 on train with SelectKBest
Test R^2 with RFE: 0.576 and 0.602 R^2 on test with SelectKBest


Качество модели на тесте МАЕ показало погрешность в 4708 долларов при определении цены, считаю это много, для модели.

Качество модели на тесте R^2, показало 0.602, не плохо, но не достаточно для хорошей модели.

Выбранные метрики показывают лучший результат на тестовой выборке.

Метод отбора SelectKBest показал лучший результат относительно RFE.